In [8]:
import psycopg2
import gdal
import shapely
import geopandas
from osgeo import ogr
from osgeo import osr
import os, subprocess



In [ ]:
#local settings
local_postgresql_path = r';C:\Program Files\PostgreSQL\13\bin'
host = 'localhost'
port = '5432'
user = 'my_username'
password = 'my_password'
database = 'my_database'

In [ ]:
#load counties shp to postgreSQL
#save off: select * from counties where statefps = '15' 
#polygonize landcover raster (to shp)
#load landcover shp to postgreSQL
#intersect landcover shp with HI counties (state fips 15)
#sum area of developed area, high intensity (class 24) within each county
#divide dev area by county area for percent of highly developed land area

In [10]:
def load_shp(shape, srid, tablename):
    driver = ogr.GetDriverByName('ESRI Shapefile')
    cmds = 'shp2pgsql -s ' + srid + ' -d "' + shape + '" ' + tablename + ' | psql '
    subprocess.call(cmds, shell=True)
    print(cmds)

In [11]:
os.environ['PATH'] += local_postgresql_path
os.environ['PGHOST'] = host
os.environ['PGPORT'] = port
os.environ['PGUSER'] = user
os.environ['PGPASSWORD'] = password
os.environ['PGDATABASE'] = database

shape_path = r"C:/Users/cstanton/Documents/Personal/DBMS/FinalProject/TIGER_counties/tl_2021_us_county.shp"
load_shp(shape_path, '4269', 'counties')

shp2pgsql -s 4269 -d "C:/Users/cstanton/Documents/Personal/DBMS/FinalProject/TIGER_counties/tl_2021_us_county.shp" counties | psql 


In [5]:
raster = gdal.Open(r"C:/Users/cstanton/Documents/Personal/DBMS/FinalProject/HI_NLCD/hi_landcover_wimperv_9-30-08_se5.img")
srcband = raster.GetRasterBand(1)

prj = raster.GetProjection()
srs=osr.SpatialReference(wkt=prj)
dst_layername = 'polygonized_test.shp'
drv = ogr.GetDriverByName("ESRI Shapefile")
dst_ds = drv.CreateDataSource(dst_layername)
dst_layer = dst_ds.CreateLayer(dst_layername, srs=srs)
fd = ogr.FieldDefn("DN", ogr.OFTInteger)
dst_layer.CreateField(fd)
dst_field = dst_layer.GetLayerDefn().GetFieldIndex("DN")
gdal.Polygonize(srcband, None, dst_layer, dst_field, [], callback=None)
print("Raster vectorized")

PROJCS["Albers Conical Equal Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",3],PARAMETER["longitude_of_center",-157],PARAMETER["standard_parallel_1",8],PARAMETER["standard_parallel_2",18],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["meters",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Albers Conical Equal Area
WGS 84


In [18]:
shape_path = r"C:/Users/cstanton/Documents/Personal/DBMS/FinalProject/polygonized_test.shp"
load_shp(shape_path,'4269','landcover_HI')

shp2pgsql -s 4269 -d "C:/Users/cstanton/Documents/Personal/DBMS/FinalProject/polygonized_test.shp" landcover_HI | psql 
